In [93]:
from quantrocket import get_prices
import pandas as pd
import numpy as np

In [88]:
# Step 1: Fetch daily close prices of Apple stock for the year 2023
start_date = '2007-01-01'
end_date = '2023-12-31'
prices = get_prices("usstock-free-1d", universes="usstock-free", start_date=start_date, end_date=end_date, fields=["Close"])
close_prices = prices['FIBBG000B9XRY4']

print('\n',apple_data)


                   Open        High         Low       Close   Adj Close  \
Date                                                                     
2020-01-02   74.059998   75.150002   73.797501   75.087502   73.059441   
2020-01-03   74.287498   75.144997   74.125000   74.357498   72.349152   
2020-01-06   73.447502   74.989998   73.187500   74.949997   72.925629   
2020-01-07   74.959999   75.224998   74.370003   74.597504   72.582642   
2020-01-08   74.290001   76.110001   74.290001   75.797501   73.750244   
...                ...         ...         ...         ...         ...   
2020-12-23  132.160004  132.429993  130.779999  130.960007  128.523270   
2020-12-24  131.320007  133.460007  131.100006  131.970001  129.514465   
2020-12-28  133.990005  137.339996  133.509995  136.690002  134.146622   
2020-12-29  138.050003  138.789993  134.339996  134.869995  132.360489   
2020-12-30  135.580002  135.990005  133.399994  133.720001  131.231903   

               Volume  
Date       

In [89]:
# Step 2: Calculate daily returns and classify states
returns = close_prices.pct_change()
print(returns)
states = np.where(returns >= 0.1, 1, np.where(returns > -0.1, 0, -1))
print(states)

Field  Date      
Close  2007-01-03         NaN
       2007-01-04    0.022169
       2007-01-05   -0.007113
       2007-01-08    0.004945
       2007-01-09    0.083069
                       ...   
       2023-12-22   -0.005547
       2023-12-26   -0.002841
       2023-12-27    0.000518
       2023-12-28    0.002226
       2023-12-29   -0.005424
Name: FIBBG000B9XRY4, Length: 4278, dtype: float64
[-1  0  0 ...  0  0  0]


In [90]:
# Step 3: Implement the value function
portfolio_value = 0
state_transitions = np.zeros((3, 3))
optimal_buy_indices = []

for i in range(1, len(states)):
    if states[i] == 1 and states[i-1] == 0:
        portfolio_value += 1
        optimal_buy_indices.append(i-1)
    elif states[i] == -1 and states[i-1] == 0:
        portfolio_value -= 1

    state_transitions[states[i-1] + 1, states[i] + 1] += 1

In [91]:
# Step 4: Calculate transition distribution
transition_distribution = np.zeros_like(state_transitions)
# Calculate transition distribution, handling division by zero
for i in range(len(state_transitions)):
    row_sum = np.sum(state_transitions[i])
    if row_sum != 0:
        transition_distribution[i] = state_transitions[i] / row_sum
print(transition_distribution)

[[0.00000000e+00 1.00000000e+00 0.00000000e+00]
 [7.03234880e-04 9.97890295e-01 1.40646976e-03]
 [1.66666667e-01 8.33333333e-01 0.00000000e+00]]


In [92]:
print("Portfolio Value:", portfolio_value)
print("Optimal Index:",optimal_buy_indices)
print("Transition Distribution:")
print(pd.DataFrame(transition_distribution, index=['Bear', 'Flat', 'Bull'], columns=['Bear', 'Flat', 'Bull']))

Portfolio Value: 3
Optimal Index: [217, 447, 477, 3320, 3327, 3417]
Transition Distribution:
          Bear      Flat      Bull
Bear  0.000000  1.000000  0.000000
Flat  0.000703  0.997890  0.001406
Bull  0.166667  0.833333  0.000000
